# September 1, 6-9: Emoprox2 dataset: MAX ROIs: create ROI-masks and trial level responses

In [1]:
import os 
import sys
from os.path import join as pjoin
import numpy as np
import scipy as sp
import pandas as pd
from tqdm.notebook import tqdm
import pickle, random
import csv

from nltools import mask
from nilearn import image, masking

# plotting
import matplotlib.pyplot as plt
plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import cmasher as cmr #CITE ITS PAPER IN YOUR MANUSCRIPT

# main dirs
proj_dir = pjoin(os.environ['HOME'], 'network_dynamics')
results_dir = f"{proj_dir}/results"
month_dir = f"{proj_dir}/nb/sep22"

# folders
sys.path.insert(0, proj_dir)
import helpers.dataset_utils as dataset_utils

/home/govindas/venvs/expln-ai3.9/lib/python3.9/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


In [2]:
'''
trial level responses
detranding and cleaning motion
'''
# run this on bswift and copy all the subject wise folders to ada
# sbatch /home/govindas/network_dynamics/data/emoprox2/scripts/01-3dDeconvolve_3dREMLfit.sh

'''
proximity stimulus
Joyneel calculated proximity values at each TR afresh, so taking from his dataframe
'''
with open(
    f"/home/joyneelm/FORAGING/local/notebooks/dynamical_systems_emoprox/emoprox_motion_parameters.pkl", 
    'rb'
) as f:
    motion_df = pickle.load(f)
motion_df

,pid,rid,timeseries,proximity,direction,speed,continuous_states,discrete_states
0,001,0,"[[-0.08312777714490793, -0.9843746554254497, 0...","[0.3016038203344943, 0.358338183215815, 0.4166...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0...","[0.0, 0.001533361158954638, 0.0015333611589546...","[[-0.7020833319593689, -0.18291312230527457, 1...","[7, 7, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,001,1,"[[0.5425140981177836, -0.879907230649685, 0.18...","[0.45156998293255135, 0.40446652419199125, 0.3...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0...","[0.0, 0.0012730664524476243, 0.001273066452447...","[[0.07061164551667026, 1.2396794139298464, -0....","[7, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 6, ..."
2,001,2,"[[1.059156933419393, 0.6198138745769199, 0.879...","[0.49544710094463773, 0.4414182695332566, 0.38...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0,...","[0.0, 0.0014602386867941242, 0.002738264303754...","[[-0.7816242821088368, 1.804785642947887, -0.3...","[7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, ..."
3,001,3,"[[0.13562674131522262, -0.16214955653579818, 0...","[0.34734037008548524, 0.3981108619589484, 0.45...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0...","[0.0, 0.001372175456039515, 0.0013721754560396...","[[-1.1839740949248894, -0.28692882635832284, 1...","[7, 7, 6, 6, 6, 6, 6, 6, 6, 3, 2, 2, 2, 6, 7, ..."
4,001,4,"[[0.17779630837305302, -0.6310155905999884, 1....","[0.49953186000240013, 0.4425583650437417, 0.38...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0,...","[0.0, 0.0015398241880717478, 0.002842105474835...","[[0.14725308336753923, 0.7553325361724021, -0....","[7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ..."
...,...,...,...,...,...,...,...,...
711,125,1,"[[0.09050368355505174, -0.04549253755032044, 0...","[0.45156998293255135, 0.40446652419199125, 0.3...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0...","[0.0, 0.0012730664524476243, 0.001273066452447...","[[0.5889450195142745, -0.4395345974608756, -0....","[7, 7, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, ..."
712,125,2,"[[0.1868332209186578, 0.809608741033865, 0.359...","[0.49544710094463773, 0.4414182695332566, 0.38...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0,...","[0.0, 0.0014602386867941242, 0.002738264303754...","[[-0.6084148492157283, 0.9691923055191999, 0.7...","[7, 7, 3, 3, 3, 4, 7, 7, 7, 6, 6, 6, 6, 5, 5, ..."
713,125,3,"[[-0.41030877310596003, 0.4596794647633776, 0....","[0.34734037008548524, 0.3981108619589484, 0.45...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0...","[0.0, 0.001372175456039515, 0.0013721754560396...","[[-0.16214038989837437, -0.5040720709742246, 1...","[7, 3, 3, 3, 3, 4, 4, 4, 7, 7, 7, 2, 2, 2, 2, ..."
714,125,4,"[[0.0068114147222914185, 0.41750824672533293, ...","[0.49953186000240013, 0.4425583650437417, 0.38...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0,...","[0.0, 0.0015398241880717478, 0.002842105474835...","[[-0.1155816806011225, -0.888793983571116, 0.3...","[7, 7, 7, 7, 7, 7, 7, 7, 3, 3, 3, 7, 7, 6, 6, ..."


In [3]:
class ARGS(): pass
args = ARGS()

args.APPR = 1.0
args.RETR = -1.0
args.LABEL_NAMES = ['RETR', 'APPR']
args.LABELS = [args.RETR, args.APPR]

# arbitrary threshold on proximity
args.near_miss_thresh = 0.7
args.near_miss_width = 8

set_name = 'max'

args.main_data_path = (
    f"{proj_dir}/data/emoprox2/data/rois_{set_name}"
)
# os.system(f"mkdir -p {main_output_path}")

args.SEED = 74
np.random.seed(args.SEED)

args.subjects = motion_df.pid.unique()
random.Random(args.SEED).shuffle(args.subjects)

'''
exploratory dataset
'''
args.explor_subjects = args.subjects[ : round(0.25 * len(args.subjects))]

X = dataset_utils.get_emo2_trial_level_responses(
    args, 
    motion_df,
    args.explor_subjects,
)
# for idx in X.keys():
#     print(len(X[idx]))

with open(f"{proj_dir}/data/emoprox2/data/exploratory_data/rois-{set_name}_exploratory_data.pkl", 'wb') as f:
    pickle.dump(X, f)

100%|██████████| 30/30 [00:01<00:00, 19.65it/s]


In [4]:
# fig = plt.subplots(figsize=(30, 7))
# plt.plot(proximity)
# plt.plot(peaks, proximity[peaks], 'x')
# plt.vlines(np.round(props['left_ips']), ymin=0, ymax=1, color='r')
# plt.vlines(np.round(props['right_ips']), ymin=0, ymax=1, color='g')